# Nabla

Automatic differentiation in python using dual numbers.

In [1]:
import nabla

def f(x):
    return 3*x*x

print(f(5))

print(nabla.grad()(f)(5))

75
Dual(75,  [30])


## Nabla supports multiple variables

In [2]:
def f(x, y, param, z):
    return 2*x*y + z**4

x, y, z, param = 1, 2, 3, "this is a non-numeric parameter"

print(f(x, y, param, z))

print(nabla.grad()(f)(x, y, param, z))

85
Dual(85,  [4, 2, 108])
